In [134]:
import pandas as pd
import numpy as npause
from scipy.stats import spearmanr

In [136]:
df = pd.read_csv(
    "2157-Generated-Data/Clinical/Regular/2157-Clinical-Biomarkers.csv",
    index_col="winterlight_id",
)
df = df.dropna(subset=["Age", "Sex", "UPDRSIII", "Speech"])
df = df.drop(["Weight", "Sex", "Age", "UPDRSIII"], axis=1)
df = df.dropna()
df["subject"] = df.subjid.str.split("O").str[0]

In [137]:
audio_file_info_path = "/Users/thomas.cong/Downloads/!RAW-RTS/Audio Files/regeneron_winterlight_data_export_2025-05-01-05-01-2025_21_51_37.csv"
info_2157 = pd.read_csv(audio_file_info_path, index_col="winterlight_sample_id")
id_to_task_mapping = {
    id: task for id, task in zip(info_2157.index, info_2157.task_name)
}
df["task"] = df.index.map(id_to_task_mapping)

In [138]:
import pandas as pd

# Assuming your DataFrame is named 'df'
# If 'winterlight_id' is the index, reset it to be a column
df_for_pivot = df.reset_index()

# Define the columns that are identifiers and not features
identifier_columns = ["winterlight_id", "subjid", "Speech", "Status", "subject", "task"]

# Get the list of feature columns by excluding the identifiers
feature_columns = [col for col in df_for_pivot.columns if col not in identifier_columns]

# Pivot the DataFrame
pivoted_df = df_for_pivot.pivot_table(
    index=["subject", "Status"], columns="task", values=feature_columns
)

# Collapse the multi-level columns into a single level
pivoted_df.columns = ["_".join(col).strip() for col in pivoted_df.columns.values]
# Unstack the 'Status' level of the index.
# This moves 'ON' and 'OFF' from the index to become column levels.
by_status_df = pivoted_df.unstack(level="Status")

# Calculate the difference: ON - OFF
# This subtracts the 'OFF' values from the 'ON' values for each feature-task combination.
# The .xs() method is used to select data at a particular level of a MultiIndex.
on_off_diff_df = by_status_df.xs("ON", level="Status", axis=1) - by_status_df.xs(
    "OFF", level="Status", axis=1
)
# Display the updated DataFrame
on_off_diff_df

,aavs_ddk,aavs_paragraph_reading,aavs_phonemic_fluency,aavs_picture_description,aavs_semantic_fluency,aavs_sustained_vowel_phonation,articulation_rate_ddk,articulation_rate_paragraph_reading,articulation_rate_phonemic_fluency,articulation_rate_picture_description,...,shimmer_local_db_phonemic_fluency,shimmer_local_db_picture_description,shimmer_local_db_semantic_fluency,shimmer_local_db_sustained_vowel_phonation,speech_rate_ddk,speech_rate_paragraph_reading,speech_rate_phonemic_fluency,speech_rate_picture_description,speech_rate_semantic_fluency,speech_rate_sustained_vowel_phonation
subject,,,,,,,,,,,,,,,,,,,,,
CU001,18644.481363,11818.061256,4039.107066,26068.629804,2091.908036,NaN,-0.472329,-1.104324,-2.203721,-0.807234,...,-0.453358,-0.049408,0.007599,NaN,-0.472329,-0.510312,0.054634,0.957745,2.106468,NaN
CU002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CU003,-14883.073725,9900.572681,NaN,63359.569883,64707.794404,NaN,0.276823,-0.056155,NaN,-0.710468,...,NaN,-0.470829,-0.496950,NaN,0.276823,-0.056155,NaN,-0.891417,-3.144828,NaN
CU004,NaN,-9379.547933,8137.677988,-861.169881,9616.136400,NaN,NaN,0.430820,0.717216,0.975564,...,-0.177297,-0.099208,-0.243106,NaN,NaN,0.430820,0.717216,0.975564,0.840687,NaN
CU005,44150.652074,48003.976507,-4681.476055,4298.264505,10074.870662,NaN,-0.262771,-1.166029,-0.717110,-0.760170,...,0.084649,0.031076,0.111567,NaN,-0.262771,-1.166029,-0.717110,-0.760170,-0.672112,NaN
CU006,NaN,65417.832927,12556.772979,63238.273044,28037.376857,NaN,NaN,-0.025353,-2.001301,0.109054,...,0.238565,0.024316,-0.007622,NaN,NaN,-0.025353,-1.866674,0.109054,-0.201819,NaN
CU007,NaN,NaN,NaN,8493.265449,-30418.889584,7098.461945,NaN,NaN,NaN,-0.621982,...,NaN,0.297184,-0.281382,-0.495220,NaN,NaN,NaN,-0.621982,-1.129533,0.980014
CU008,26220.546589,19813.315989,3456.379242,12591.309585,17003.906374,18994.984190,-0.157128,0.238486,0.677664,0.121509,...,-0.183319,-0.127326,-0.078591,-0.044284,-0.209387,-0.390978,-0.756066,-0.268817,0.656267,-0.246217
CU009,NaN,-29567.059585,6180.488051,-9456.870216,12189.699985,NaN,NaN,0.708965,0.782609,0.086812,...,0.004440,0.199831,0.001277,NaN,NaN,-0.423175,0.937524,1.792365,0.479316,NaN


In [139]:
from scipy.stats import ttest_1samp

# Assuming 'on_off_diff_df' is your DataFrame of ON-OFF differences

# --- Create the ranked list for GSEA ---

# We will calculate the t-statistic for each feature column.
# This is done by performing a one-sample t-test on the differences,
# testing if the mean difference is significantly different from zero.
# We use .dropna() to handle cases where some subjects may have NaN values.

p_values = []
feature_names = []

for feature in on_off_diff_df.columns:
    # Get the difference values for the feature, dropping NaNs
    diff_values = on_off_diff_df[feature].dropna()

    # Ensure there's enough data to run the test
    if len(diff_values) > 1:
        # Calculate the t-statistic
        t_stat, p_val = ttest_1samp(diff_values, 0)
        p_values.append(p_val)
        feature_names.append(feature)

# Create a pandas Series with features and their t-statistics
ranked_features = pd.Series(data=p_values, index=feature_names)

# Sort the features in descending order to create the final ranked list
ranked_features.sort_values(ascending=True, inplace=True)


# --- Display the result ---
print("GSEA Ranked Feature List (top 10):")
print(ranked_features.head(10))

print("\n(bottom 10):")
print(ranked_features.tail(10))

GSEA Ranked Feature List (top 10):
hull_area_picture_description           0.001131
harm_mean_sustained_vowel_phonation     0.002749
harm_mean_paragraph_reading             0.003948
inten_max_paragraph_reading             0.004101
inten_min_paragraph_reading             0.005760
inten_mean_paragraph_reading            0.006493
harm_std_paragraph_reading              0.007682
inten_median_paragraph_reading          0.008447
content_richness_picture_description    0.010156
ff_std_picture_description              0.011331
dtype: float64

(bottom 10):
cpp_semantic_fluency                  0.924222
shimmer_apq3_picture_description      0.937650
harm_std_picture_description          0.949731
shimmer_local_db_ddk                  0.952222
harm_std_phonemic_fluency             0.959507
shimmer_apq5_picture_description      0.960046
harm_max_picture_description          0.960051
sentence_length_phonemic_fluency      0.972201
harm_std_sustained_vowel_phonation    0.978782
inten_max_picture_descr

In [ ]:
task_sets = {
    "picture_description": [
        x for x in on_off_diff_df.columns if "picture_description" in x
    ],
    "paragraph_reading": [
        x for x in on_off_diff_df.columns if "paragraph_reading" in x
    ],
    "semantic_fluency": [x for x in on_off_diff_df.columns if "semantic_fluency" in x],
    "sustained_vowel_phonation": [
        x for x in on_off_diff_df.columns if "sustained_vowel_phonation" in x
    ],
    "ddk": [x for x in on_off_diff_df.columns if "ddk" in x],
    "phonemic_fluency": [x for x in on_off_diff_df.columns if "phonemic_fluency" in x],
}
try:
    df = df.drop(labels=["subjid", "Speech", "Status", "subject", "task"], axis=1)
except:
    pass
feature_sets = {
    feature: [x for x in on_off_diff_df.columns if feature in x]
    for feature in df.columns
}
articulatory = [
    "aavs",
    "hull_area",
    "articulation_rate",
    "speech_rate",
    "avg_pause_duration",
]
cognitive = [
    "avg_word_length",
    "content_richness",
    "mattr",
    "phrase_patterns",
    "sentence_length",
]
phonatory = ["ff", "harm", "inten", "jitter", "shimmer", "ppe", "cpp"]

types = {
    "Articulatory": [x for x in on_off_diff_df.columns if any(sub in x for sub in articulatory)],
    "Cognitive": [x for x in on_off_diff_df.columns if any(sub in x for sub in cognitive)],
    "Phonatory": [x for x in on_off_diff_df.columns if any(sub in x for sub in phonatory)]
}

groups = { **types, **task_sets}

In [141]:
import gseapy as gp

ranked_df = ranked_features.reset_index().rename(
    columns={"index": "feature", "rank": "pvalue"}
)
gsea_results = gp.prerank(
    rnk=ranked_df,
    gene_sets=groups,
    threads=4,
    permutation_num=1000,
    outdir="GSEA Results",
    seed=666,
    format="png",
)

In [142]:
results_df = gsea_results.res2d

In [143]:
results_df

,Name,Term,ES,NES,NOM p-val,FDR q-val,FWER p-val,Tag %,Gene %,Lead_genes
0,prerank,picture_description,0.48612,1.471998,0.021021,0.113151,0.108,22/35,41.90%,inten_max_picture_description;harm_max_picture...
1,prerank,sustained_vowel_phonation,0.410051,1.237751,0.116,0.549733,0.742,15/35,36.19%,harm_std_sustained_vowel_phonation;ff_max_sust...
2,prerank,semantic_fluency,0.410514,1.235541,0.122244,0.374166,0.754,23/35,46.67%,cpp_semantic_fluency;harm_std_semantic_fluency...
3,prerank,Cognitive,0.375335,1.110648,0.289289,0.659379,0.987,8/30,11.90%,sentence_length_phonemic_fluency;avg_word_leng...
4,prerank,phonemic_fluency,0.332973,1.015487,0.468938,0.844927,1.0,24/35,55.71%,sentence_length_phonemic_fluency;harm_std_phon...
5,prerank,Articulatory,0.309731,0.908181,0.660661,0.996829,1.0,12/30,39.52%,speech_rate_picture_description;hull_area_para...
6,prerank,Phonatory,0.262802,0.839229,0.804,0.999189,1.0,84/138,60.00%,inten_max_picture_description;harm_std_sustain...
7,prerank,ddk,0.237895,0.726457,0.906,1.0,1.0,24/35,63.33%,shimmer_local_db_ddk;sentence_length_ddk;mattr...
8,prerank,paragraph_reading,0.191366,0.576675,0.98794,0.985314,1.0,6/35,20.48%,average_syllable_duration_paragraph_reading;cp...
